<a href="https://colab.research.google.com/github/ugursirvermez/PyTorch_Education/blob/main/07_python_script_module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Script Hale Getirme

#Data_Setup.py

In [14]:
%%writefile data_setup.py
#going_modular adında olusturdugumuz dosyanın üzerine bu kodu data_setup.py olarak yaz.
import os
import zipfile
from pathlib import Path
import requests

import torch
#Torchvision Kutuphaneleri
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader

#-------------------------------------------------------------------------------------------------------------
#VERİ SETLERİ

# Dosyaların cikacagi yolu ayarla
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

# Eger dosya yoksa indirmeye basla
if image_path.is_dir():
    print(f"{image_path} dosya zaten var.")
else:
    print(f"{image_path} Dosyası olusturuluyor")
    image_path.mkdir(parents=True, exist_ok=True)

# Yemek bilgilerini indir
with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("İndiriliyor...")
    f.write(request.content)

# Dosyaları zipten cikar
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
    print("Sıkıştırılmış dosyada çıkarılıyor...")
    zip_ref.extractall(image_path)

# Zip dosyasini sil.
os.remove(data_path / "pizza_steak_sushi.zip")

# train ve test degiskenlerine dosyalari at.
train_dir = image_path / "train"
test_dir = image_path / "test"

# Transform ile verileri donustur.
data_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

# ImageFolder'dan bir train_data olustur. Data Load etme işlemi
train_data = datasets.ImageFolder(root=train_dir, # hedeflenen görseller
                                  transform=data_transform, # data_transform şekline çevir
                                  target_transform=None) # Etikete özel dnosutrme yapılmayacak.

#Test_data benzer şekilde olusturuldu.
test_data = datasets.ImageFolder(root=test_dir,
                                 transform=data_transform)

print(f"Train data:\n{train_data}\nTest data:\n{test_data}") #veriler geldi mi?
#--------------------------------------------------------------------------------------------------------

#DATA_SETUP.PY BURADA BASLIYOR
#İşlemci sayilari
NUM_WORKERS = os.cpu_count()

#DataLoader Fonksiyonu DataSet -> DataLoader
#Train, test ve sınıfların etiketlerini deger olarak dondurecek bir fonksiyondur.
def create_dataloaders(
    train_dir: str, #Train Klasoru
    test_dir: str, #Test Klasoru
    transform: transforms.Compose, #transform dosyalarını bir araya getir. (train ve test)
    batch_size: int, #DataLoader'dan ne kadar batch yapılacak?
    num_workers: int=NUM_WORKERS #DataLoader'da kaç işlemci çalışacak?
):
   # ImageFolder Kullanarak dataset olusturma
  train_data = datasets.ImageFolder(train_dir, transform=transform)
  test_data = datasets.ImageFolder(test_dir, transform=transform)

  # Train_data'da etiketlenmiş verileri al.
  class_names = train_data.classes

  # Resimleri DataLoader'a donustur
  train_dataloader = DataLoader(train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
  )
  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False,
      num_workers=num_workers,
      pin_memory=True,
  )

  return train_dataloader, test_dataloader, class_names #Loader dosyaları ve etiketler dondursun.

#data_setup.py ile dosyaları DataLoader'ı kullanabiliriz.

Overwriting data_setup.py


#Model_Builder.py

In [15]:
#Model_Builder.py inşa etme
%%writefile model_builder.py
import torch
from torch import nn

class TinyVGG(nn.Module):
  # 4 Conv2d Model ve son olarak classifier olarak çalışan model
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
      super().__init__()
      self.conv_block_1 = nn.Sequential(
          nn.Conv2d(in_channels=input_shape,
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=0),
          nn.ReLU(),
          nn.Conv2d(in_channels=hidden_units,
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=0),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,
                        stride=2)
      )
      self.conv_block_2 = nn.Sequential(
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.MaxPool2d(2)
      )
      self.classifier = nn.Sequential(
          nn.Flatten(),
          # in_features nereden geldiğine bakılmalı.
          # Kurulan her ağ modelimizi genişletiyor.
          #Neden 13x13?
          nn.Linear(in_features=hidden_units*13*13,
                    out_features=output_shape)
      )

  def forward(self, x: torch.Tensor):
    x = self.conv_block_1(x)
    x = self.conv_block_2(x)
    x = self.classifier(x)
    return x
    # return self.classifier(self.conv_block_2(self.conv_block_1(x)))



Overwriting model_builder.py


#Engine.py

In [16]:
#Engine.py oluşturduk.
%%writefile engine.py
import torch

from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train_step(model: torch.nn.Module, dataloader: torch.utils.data.DataLoader,loss_fn: torch.nn.Module, optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:

  # modeli eğitmeye başla
  model.train()

  # loss ve accuracy yani kayıp ve tutarlılık miktarı
  train_loss, train_acc = 0, 0


  # Donguyu dataloader'dan çalıştır
  for batch, (X, y) in enumerate(dataloader):
      # device ? cpu : gpu -> cihaza yolla
      X, y = X.to(device), y.to(device)

      # 1.Forward etme
      y_pred = model(X)

      # 2. Kaybı hesaplama
      loss = loss_fn(y_pred, y)
      train_loss += loss.item()

      # 3. Optimizer zero grad
      optimizer.zero_grad()

      # 4. Loss backward
      loss.backward()

      # 5. Optimizer step
      optimizer.step()

      # Batch'teki değerleri kümeleyerek hesapla.
      y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
      train_acc += (y_pred_class == y).sum().item()/len(y_pred)

  # Ortalama kayıp ve tutarlılığı hesapla.
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc

def test_step(model: torch.nn.Module, dataloader: torch.utils.data.DataLoader, loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:

  # Modeli Değerlendir
  model.eval()

  # test kayıp ve tutarlılıkları hesaplayacağız
  test_loss, test_acc = 0, 0
  # İçeriği kontrol edeceğiz.
  with torch.inference_mode():
      # DataLoader'daki Batch'i yükle
      for batch, (X, y) in enumerate(dataloader):
          # device ? cpu : gpu -> cihaza yolla
          X, y = X.to(device), y.to(device)

          # 1. Forward et
          test_pred_logits = model(X)

          # 2. loss ve acc hesapla
          loss = loss_fn(test_pred_logits, y)
          test_loss += loss.item()

          test_pred_labels = test_pred_logits.argmax(dim=1) #logaritmadan çevir.
          test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

  # ortalama test loss ve acc'ı yazdır.
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc

#TRAIN_STEP VE TEST_STEP ILE DONGUYU CALISTIR!
def train(model: torch.nn.Module, train_dataloader: torch.utils.data.DataLoader, test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer, loss_fn: torch.nn.Module, epochs: int, device: torch.device) -> Dict[str, List]:
  # İki step'teki sonuçları yazdır.
  results = {"train_loss": [],
      "train_acc": [],
      "test_loss": [],
      "test_acc": []
  }

  # train ve test step'lerini döngüyle çalıştır.
  for epoch in tqdm(range(epochs)):
      #Train_Step kısmı
      train_loss, train_acc = train_step(model=model,dataloader=train_dataloader, loss_fn=loss_fn, optimizer=optimizer, device=device)
      #Test_Step kısmı
      test_loss, test_acc = test_step(model=model, dataloader=test_dataloader,loss_fn=loss_fn, device=device)

      # Neler olduğunu adım adım yazdır.
      print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
      )

      # Güncellenen sonuçları yazdir
      results["train_loss"].append(train_loss)
      results["train_acc"].append(train_acc)
      results["test_loss"].append(test_loss)
      results["test_acc"].append(test_acc)

  # Toplam sonucu ver.
  return results

Overwriting engine.py


#Utils.py

In [17]:
#utils.py ile save model yapıyoruz.
%%writefile utils.py

import torch
from pathlib import Path

#Modeli Kaydetme İşlemi
def save_model(model: torch.nn.Module, target_dir: str, model_name: str):
   #Hedeflenen klasörü ve yolu oluşturma
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True, exist_ok=True)

  #Modelin kaydedileceği yolu oluşturma
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name sonunda '.pt' veya '.pth' ile bitmelidir!"
  model_save_path = target_dir_path / model_name

  #state_dict() ile son durumu güncelleme
  print(f"[INFO] Modelin Kaydedildiği Yer: {model_save_path}")
  torch.save(obj=model.state_dict(), f=model_save_path)

Overwriting utils.py


#Train.py Oluşturma

In [18]:
#Train.py Dosyasını Oluşturduk.
%%writefile train.py

import os
import torch
import data_setup, engine, model_builder, utils #Bütün oluşturduğumuz dosyaları alıyoruz.
from torchvision import transforms

# Girdi Parametrelerimizi oluşturuyoruz. -> HiperParametre
NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001

# Klasörleri hazırladık.
train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"

# Cihaz CPU mu yoksa GPU mu buna karar veriyoruz.
device = "cuda" if torch.cuda.is_available() else "cpu"

#Transform Oluşturma
data_transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor()])

#data_setup.py ile DataLoader oluşturma
#3 Parametre döndürüyordu. train ve test dataloader ile etiketlerdi.
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir, test_dir=test_dir,
                                                                               transform=data_transform,batch_size=BATCH_SIZE) #HiperParametre var.

#model_builder.py ile birlikte modeli oluşturuyoruz.
model = model_builder.TinyVGG(input_shape=3, hidden_units=HIDDEN_UNITS,output_shape=len(class_names)).to(device)#HiperParametre var.

#loss ve optimizer hazırlama
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=LEARNING_RATE) #HiperParametre var.

# engine.py ile eğitimi başlatabiliriz.
engine.train(model=model, train_dataloader=train_dataloader,test_dataloader=test_dataloader,loss_fn=loss_fn, optimizer=optimizer,
             epochs=NUM_EPOCHS, device=device) #HiperParametre var.

#utils.py ile modelin son halini kaydetme
utils.save_model(model=model, target_dir="models", model_name="07_modular_script_mode_tinyvgg_model.pth")

Overwriting train.py


#Kodları Çalıştırma

In [19]:
!python train.py

data/pizza_steak_sushi Dosyası olusturuluyor
İndiriliyor...
Sıkıştırılmış dosyada çıkarılıyor...
Train data:
Dataset ImageFolder
    Number of datapoints: 225
    Root location: data/pizza_steak_sushi/train
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )
Test data:
Dataset ImageFolder
    Number of datapoints: 75
    Root location: data/pizza_steak_sushi/test
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )
  0% 0/5 [00:00<?, ?it/s]Epoch: 1 | train_loss: 1.1066 | train_acc: 0.2578 | test_loss: 1.0873 | test_acc: 0.2604
 20% 1/5 [00:02<00:08,  2.21s/it]Epoch: 2 | train_loss: 1.1050 | train_acc: 0.3125 | test_loss: 1.0644 | test_acc: 0.5729
 40% 2/5 [00:04<00:05,  1.99s/it]Epoch: 3 | train_loss: 1.0784 | train_acc: 0.3906 | test_loss: 1.0790 |